In [27]:
#### where to see json in good format http://json.parser.online.fr/

import pandas as pd
import json

inpath = "E:/OneDrive/Datasets/Tuboleta/Hubspot/"
#inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Hubspot/"

tb_ini = pd.read_csv(inpath + 'hubspot_json.csv',delimiter=';')
tb_ini.fillna(0, inplace=True)

print('Dimensiones:')
print(tb_ini.shape)
print(tb_ini.head(5))
print(' ')
print('Summary:')
tb_ini.info()

Dimensiones:
(131150, 4)
   id                                              trama  \
0   1  [{"orderDetailsData":{"orderId":10000702652,"o...   
1   2  [{"orderDetailsData":{"orderId":10000702652,"o...   
2   3  {"orderDetailGranularity":"TICKET","orderDetai...   
3   4  {"orderDetailGranularity":"TICKET","orderDetai...   
4   5  {"orderDetailGranularity":"TICKET","orderDetai...   

                     fecha  procesado  
0  2023-11-30 01:46:04.463          0  
1  2023-11-30 15:05:45.197          0  
2  2023-11-30 15:10:54.960          0  
3  2023-11-30 15:10:55.113          0  
4  2023-11-30 15:10:55.270          0  
 
Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131150 entries, 0 to 131149
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         131150 non-null  int64 
 1   trama      131150 non-null  object
 2   fecha      131150 non-null  object
 3   procesado  131150 non-null  int64 
dtypes: int64

In [78]:
tb = tb_ini#.tail(5)
print(tb.shape)
#print(tb.head(5))

(131150, 6)


In [59]:
# FUNCION QUE PERMITE EXTRAER DOS ELEMENTOS DE UN ARCHIVO JSON ANIDADO amo a adri
def extract_from_json(json_string):
    try:
        json_data = json.loads(json_string)
    except (json.JSONDecodeError, TypeError):
        json_data = json_string
    
    order_state = None
    contact_reference = None

    if isinstance(json_data, list):
        for item in json_data:
            if isinstance(item, dict):
                if 'orderDetailsData' in item:
                    order_state = item['orderDetailsData'].get('orderState')
                    contact_reference = item['orderDetailsData'].get('contactReference')
                    break  # Detener el bucle una vez que se encuentren los valores
    elif isinstance(json_data, dict):
        if 'orderDetailsData' in json_data:
            order_state = json_data['orderDetailsData'].get('orderState')
            contact_reference = json_data['orderDetailsData'].get('contactReference')
    
    return order_state, contact_reference


# Supongamos que tienes un DataFrame llamado 'df' con una columna 'json_data' que contiene los JSON anidados como strings

# Crear una función para aplicar a cada celda de la columna 'json_data'
def extract_values(row):
    order_state, contact_reference = extract_from_json(row['trama'])
    return pd.Series({'order_state': order_state, 'contact_reference': contact_reference})



In [76]:
# Aplicar la función a cada fila del DataFrame y guardar los resultados en nuevas columnas
tb[['order_state', 'contact_reference']] = tb.apply(extract_values, axis=1)

# Excluir múltiples columnas usando drop()
cols_to_exclude = ['trama']
tb = tb.drop(cols_to_exclude, axis=1)

# Mostrar el DataFrame resultante con las nuevas columnas
print('Dimensiones:')
print(tb.shape)
print(tb.head(5))

# exportamos
tb.to_excel(inpath +'extract_from_json.xlsx', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
            ) 